# EDA

## I. Baseline

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
df = pd.read_csv('dataset/train.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.shape

(891, 12)

Проверяем данные на наличие нулевых значений. Замечаем, что признак "Cabin" на 77% состоит из пустых значений. Небольшая часть значений признаков "Age" и "Embarked" содержит нули.

In [5]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [12]:
# сразу удалим колонку 'Cobin'
df = df.drop('Cabin', axis=1)

KeyError: "['Cabin'] not found in axis"

Проверяем есть ли в данных повторяющиеся объекты. В результате выполнения кода видим, что дубликатов нет.

In [6]:
df.duplicated().any()

False

### Заполнение нулей

In [22]:
df.Age = df.Age.fillna(df.Age.median())
df.Embarked = df.Embarked.fillna(df.Embarked.mode())